In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def create_inception_module(input_tensor):
    # Define the Inception-like module with 4 parallel convolution channels
    conv1x1 = layers.Conv2D(64, (1, 1), activation='relu')(input_tensor)
    print("Success 1")
    conv3x3_reduce = layers.Conv2D(64, (1, 1), activation='relu')(input_tensor)
    print("Success 2")
    conv3x3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3x3_reduce)
    print("Success 3")
    conv5x5_reduce = layers.Conv2D(32, (1, 1), activation='relu')(input_tensor)
    print("Success 4")
    conv5x5 = layers.Conv2D(64, (5, 5), activation='relu', padding='same')(conv5x5_reduce)
    print("Success 5")
    maxpool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(conv5x5)
    print("Success 6")
    maxpool_proj = layers.Conv2D(32, (1, 1), activation='relu')(maxpool)
    print("Success 7")

    # Concatenate the outputs of the parallel channels
    inception_module = layers.concatenate([conv1x1, conv3x3, conv5x5, maxpool_proj], axis=-1)
    print("Success 8")

    return inception_module


In [ ]:


def create_siamese_branch(input_shape):
    # First, add a few convolutional layers

    '''layer1 = layers.Conv2D(32, (3, 3), activation='relu',padding = "valid")
    layer2 = layers.Conv2D(64, (3, 3), activation='relu', padding = "same")(layer1)'''
    input_layer = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu',padding = "valid")(input_layer)
    x = layers.Conv2D(32, (3, 3), activation='relu',padding = "valid")(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding = "same")(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding = "same")(x)
    x = layers.Conv2D(80, (3, 3), activation='relu', padding = "same" )(x)
    x = layers.Conv2D(80, (3, 3), activation='relu', padding = "same" )(x)
    x = layers.MaxPooling2D((3, 3))(x)

    # Add the Inception-like module here
    # print(model.layers[-1].output.shape[1:])
    x = create_inception_module(x)  # Pass the output of the last layer

    # Continue with more convolutional layers
    x = layers.Conv2D(128, (3, 3), activation='relu', padding = "same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    model = keras.Model(input_layer, [x])
    return model

In [ ]:
model = create_siamese_branch((112,112,3))

model

In [ ]:
model.summary()

In [ ]:
input_shape = (224, 224, 3)
# Define the Siamese network with two branches
branch_a = create_siamese_branch(input_shape)
branch_b = create_siamese_branch(input_shape)

# Define the inputs for the Siamese branches
input_a = keras.Input(shape=input_shape)
input_b = keras.Input(shape=input_shape)

# Get the outputs of the Siamese branches
output_a = branch_a(input_a)
output_b = branch_b(input_b)

In [ ]:
def contrastive_loss(y_true, y_pred, margin=1):
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)


In [ ]:
L1_distance = tf.abs(output_a - output_b)


In [ ]:
#Create the final output layer for your Siamese network, which predicts the similarity between the input pairs
prediction = keras.layers.Dense(1, activation='sigmoid')(L1_distance)

siamese_network = keras.Model(inputs=[input_a, input_b], outputs=prediction)


In [ ]:
#Function to create pairs 
def create_pairs(data, labels):
    pairs, y = [], []
    num_classes = max(labels) + 1
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]

    for i in range(len(data)):
        x1 = data[i]
        label1 = labels[i]
        x2 = data[np.random.choice(class_indices[label1])]
        label2 = label1
        pairs += [[x1, x2]]
        y += [1]  # Similar pair

        other_class = np.random.randint(0, num_classes)
        x2 = data[np.random.choice(class_indices[other_class])]
        label2 = other_class
        pairs += [[x1, x2]]
        y += [0]  # Dissimilar pair

    return np.array(pairs), np.array(y)

# Assuming you have a dataset and labels, you can create pairs like this
pairs, y = create_pairs(your_data, your_labels)



In [ ]:
#Train Siamese network with your data pairs and labels.

siamese_network.fit([pairs[:, 0], pairs[:, 1]], y, epochs=100, batch_size=10)

